In [ ]:
! pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.0 MB/s eta 0:00:00


In [ ]:
! pip install -q -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.1 MB/s eta 0:00:00


In [ ]:
! pip install -q transformers datasets peft git+https://github.com/huggingface/trl.git accelerate bitsandbytes packaging ninja sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive")
transformer_path = "/content/drive/My Drive/transformers/"

Mounted at /content/drive


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import random
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import numpy as np
import pandas as pd
import transformers
from accelerate import Accelerator
import bitsandbytes as bnb
from datasets import load_dataset, concatenate_datasets

In [ ]:
# HuggingFace Token
token='' # Add your token here

In [ ]:
# Load the dataset
dataset = load_dataset(
    'Amod/mental_health_counseling_conversations',
    token=token,
    cache_dir=transformer_path
)

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16 # A100
)

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer_name = "mistralai/Mistral-7B-v0.1"

#Load Tokenizer
tokenizer= AutoTokenizer.from_pretrained(
    tokenizer_name,
    token=token,
    cache_dir=transformer_path,
)

# Add Padding Token
tokenizer.pad_token = tokenizer.eos_token

# Load the model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    device_map={"": Accelerator().local_process_index},
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    cache_dir=transformer_path,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

In [ ]:
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
    output_dir=os.path.join(transformer_path, 'result'),
    overwrite_output_dir='True',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=8.0e-5,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    push_to_hub=True,
    hub_model_id="clement-cvll/mental-health-mistral7b",
    hub_token=token,
)

In [ ]:
# Lets prepare the data for the training

def format(entry):
  formatted = f"<|User|>\n{entry['Context']}{tokenizer.eos_token}\n<|Assistant|>\n{entry['Response']}{tokenizer.eos_token}\n"

  return formatted

In [ ]:
train = dataset['train'].select(range(3000))
val = dataset['train'].select(range(3000, 3512))

In [ ]:
print(format(dataset['train'][0]))

<|User|>
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?</s>
<|Assistant|>
If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terr

In [ ]:
max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format,
    neftune_noise_alpha=5,
    args=args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:234: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# train
trainer.train(resume_from_checkpoint=False)

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=940, training_loss=0.0, metrics={'train_runtime': 0.1591, 'train_samples_per_second': 18456.071, 'train_steps_per_second': 4618.731, 'total_flos': 3.297929816388403e+17, 'train_loss': 0.0, 'epoch': 5.0})

In [ ]:
# save model
trainer.save_model(os.path.join(transformer_path, 'result_mistral_7b'))

Upload 16 LFS files:   0%|          | 0/16 [00:00<?, ?it/s]

events.out.tfevents.1705417522.959a86e2d3c9.1591.0:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

events.out.tfevents.1706107170.bd156cd8079b.322.0:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

events.out.tfevents.1706094180.8480cf6e829d.1937.0:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1706094374.8480cf6e829d.1937.1:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

...

events.out.tfevents.1706195797.ffc880c2fab0.1017.0:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

events.out.tfevents.1706195896.ffc880c2fab0.1017.1:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

events.out.tfevents.1706198162.ffc880c2fab0.1017.2:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]